# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7f84785c6910>)

In [4]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'M1_Wildcard', 'ConfirmedCases',
       'ConfirmedDeaths', 'StringencyIndex', 'StringencyIndexForDisplay',
       'StringencyLegacyIndex', 'StringencyLegacyIndexForDisplay',
       'GovernmentResponseIndex', 'Gove

In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [7]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [8]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [9]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [10]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [11]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [12]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89440,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
89441,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
89442,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
89443,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,213.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [13]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [14]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [15]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [16]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=10000, positive=True, precompute=True,
      selection='random')

In [17]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 143.96072294863617
Test MAE: 147.06037606429106


In [18]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -7 NewCases 0.04383780182133295
Day -6 NewCases 0.38835038550625
Day -5 NewCases 0.26795066645359944
Day -4 NewCases 0.06055017624961843
Day -3 NewCases 0.024816828421479375
Day -2 NewCases 0.08895321464301278
Day -1 NewCases 0.19943197458860648
Day -26 C6_Stay at home requirements 5.934389586953503
Day -22 C2_Workplace closing 2.9795410868712344
Day -21 C2_Workplace closing 10.513420195418005
Intercept 26.970195244296576


In [19]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [20]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [21]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 67.88819334179738
2020-08-02: 73.1637233995244
2020-08-03: 84.95080954976001
2020-08-04: 96.29760644457569
2020-08-05: 97.85387095462863
2020-08-06: 106.45721477847292
2020-08-07: 135.76958680770292
2020-08-08: 151.2895591192481
2020-08-09: 165.14907624330607
2020-08-10: 175.8822876383444
2020-08-11: 184.82277383208378
2020-08-12: 200.10783734821467
2020-08-13: 220.9762227879765
2020-08-14: 238.3953180283373
2020-08-15: 253.5849088054375
2020-08-16: 266.77851061792927
2020-08-17: 280.49497533225224
2020-08-18: 297.7553663808854
2020-08-19: 317.10668692526184
2020-08-20: 335.39019779745865
2020-08-21: 352.21449183984174
2020-08-22: 368.1864836925478
2020-08-23: 385.02395537278454
2020-08-24: 403.8168194571238
2020-08-25: 423.6483821518853
2020-08-26: 443.11688375130797
2020-08-27: 455.9300441600354
2020-08-28: 483.8125527787333
2020-08-29: 507.45105659176943
2020-08-30: 529.4925862635175
2020-08-31: 551.2009183533467

Predicting for Afghanistan__na

2020-08-21: 332.52949033568933
2020-08-22: 337.2333797836605
2020-08-23: 349.1584329447484
2020-08-24: 366.35518012489314
2020-08-25: 384.98171899834836
2020-08-26: 402.63377188696376
2020-08-27: 405.1442753938821
2020-08-28: 414.42069215195147
2020-08-29: 427.50511883341926
2020-08-30: 444.26300164113235
2020-08-31: 461.86855427013364

Predicting for Belgium__nan
2020-08-01: 2540.588319654115
2020-08-02: 1941.3786995549303
2020-08-03: 1733.8393506677771
2020-08-04: 2206.179357524774
2020-08-05: 2110.987505744911
2020-08-06: 1644.175555307603
2020-08-07: 2234.579060562129
2020-08-08: 2160.0980980584536
2020-08-09: 2200.111627790725
2020-08-10: 2336.681315386948
2020-08-11: 2260.033647035484
2020-08-12: 2226.118547682987
2020-08-13: 2407.210984296075
2020-08-14: 2454.3558493361775
2020-08-15: 2523.25222904984
2020-08-16: 2577.879834278055
2020-08-17: 2574.171234762467
2020-08-18: 2614.8481739299714
2020-08-19: 2709.6309883944336
2020-08-20: 2780.0759839088023
2020-08-21: 2846.8012868693

2020-08-08: 221.4223356869823
2020-08-09: 249.87310642425024
2020-08-10: 269.61241971330935
2020-08-11: 272.0926765618326
2020-08-12: 274.8256543337797
2020-08-13: 297.12430452028735
2020-08-14: 320.80914208657913
2020-08-15: 345.0250722645299
2020-08-16: 362.2577851879418
2020-08-17: 372.3474337604004
2020-08-18: 385.07264134104145
2020-08-19: 405.5277665560985
2020-08-20: 428.697171808977
2020-08-21: 451.12424443295544
2020-08-22: 479.90690989370717
2020-08-23: 500.51863860275324
2020-08-24: 520.0141790804989
2020-08-25: 542.5178088818082
2020-08-26: 566.89816065013
2020-08-27: 592.9316063331883
2020-08-28: 619.7149848437266
2020-08-29: 643.8300319837463
2020-08-30: 667.6486493624402
2020-08-31: 692.9116289093404

Predicting for Brazil__nan
2020-08-01: 35247.445420322845
2020-08-02: 28844.034779582023
2020-08-03: 34882.01602560334
2020-08-04: 44507.41800354113
2020-08-05: 36477.51207077151
2020-08-06: 25528.825302665475
2020-08-07: 33022.17046947501
2020-08-08: 34602.489831750354
202

2020-08-08: 164.7521382629264
2020-08-09: 170.85929262019908
2020-08-10: 176.40230535123538
2020-08-11: 186.8710473328541
2020-08-12: 206.4618710845097
2020-08-13: 231.3974114532154
2020-08-14: 248.42306085239807
2020-08-15: 259.7405435767099
2020-08-16: 270.5426083514749
2020-08-17: 285.1938936717975
2020-08-18: 305.1370021716296
2020-08-19: 326.4755026363044
2020-08-20: 344.26031471290366
2020-08-21: 359.12330200394257
2020-08-22: 363.5101068080413
2020-08-23: 375.9680982488274
2020-08-24: 394.393425618877
2020-08-25: 414.11155567596614
2020-08-26: 432.0824819597489
2020-08-27: 434.4904041028912
2020-08-28: 443.86749584112323
2020-08-29: 457.5591502287736
2020-08-30: 475.25682613446276
2020-08-31: 493.66332597785146

Predicting for Cameroon__nan
2020-08-01: 77.39929147709643
2020-08-02: 158.19875523079386
2020-08-03: 229.39959605871013
2020-08-04: 144.7641580678032
2020-08-05: 112.06208888292682
2020-08-06: 123.56906124952405
2020-08-07: 176.87412045385005
2020-08-08: 236.44201397227


Predicting for Germany__nan
2020-08-01: 10744.424199753317
2020-08-02: 9931.32799249936
2020-08-03: 16696.95920189487
2020-08-04: 21010.160350276103
2020-08-05: 16749.260960899202
2020-08-06: 10176.601119742789
2020-08-07: 11937.888798382492
2020-08-08: 13828.010841742655
2020-08-09: 17688.039293653932
2020-08-10: 19101.631581740443
2020-08-11: 16653.672888121153
2020-08-12: 14234.110878194377
2020-08-13: 14704.950584795979
2020-08-14: 16454.003750312306
2020-08-15: 18597.10872525579
2020-08-16: 19107.528234798938
2020-08-17: 17934.85348494822
2020-08-18: 16984.68949190771
2020-08-19: 17378.758062255965
2020-08-20: 18648.76417211061
2020-08-21: 19888.229481883824
2020-08-22: 20189.10162090029
2020-08-23: 19730.087458575956
2020-08-24: 19458.154493275277
2020-08-25: 19897.290678878697
2020-08-26: 20810.103767440745
2020-08-27: 21602.43977534729
2020-08-28: 21884.273042015247
2020-08-29: 21822.269968137156
2020-08-30: 21901.848197711097
2020-08-31: 22370.38020013518

Predicting for Djib

...

2020-08-01: 1172.3799435254691
2020-08-02: 1381.192065703969
2020-08-03: 2214.512618101791
2020-08-04: 3684.9781736817467
2020-08-05: 2719.086812560486
2020-08-06: 1532.7209296913582
2020-08-07: 1650.8011480127411
2020-08-08: 1989.6612736194986
2020-08-09: 2706.6041496885373
2020-08-10: 3159.5994665096505
2020-08-11: 2700.7449683808577
2020-08-12: 2216.4118592532423
2020-08-13: 2218.303315175337
2020-08-14: 2520.5173369467975
2020-08-15: 2955.813717120428
2020-08-16: 3124.62886437187
2020-08-17: 2916.516154690835
2020-08-18: 2711.430958631184
2020-08-19: 2743.0457449008927
2020-08-20: 2970.2506505014267
2020-08-21: 3228.214031614568
2020-08-22: 3327.6287636770894
2020-08-23: 3252.56135810443
2020-08-24: 3186.295991606534
2020-08-25: 3248.6567943503105
2020-08-26: 3418.0981549368826
2020-08-27: 3582.09172930957
2020-08-28: 3657.206798853188
2020-08-29: 3652.2056741985407
2020-08-30: 3659.9041695643327
2020-08-31: 3739.5034264822843

Predicting for United States__Massachusetts
2020-08-01

2020-08-27: 1150.0620611188433
2020-08-28: 1166.284741029588
2020-08-29: 1191.6087033730778
2020-08-30: 1229.8027273538644
2020-08-31: 1271.7077087192988

Predicting for United States__Nebraska
2020-08-01: 1265.261567163049
2020-08-02: 1489.9216756710339
2020-08-03: 1886.1867996464475
2020-08-04: 1997.4350645052525
2020-08-05: 1717.5783278505478
2020-08-06: 1151.0072226145317
2020-08-07: 1489.0329623066323
2020-08-08: 1754.7313569710486
2020-08-09: 2000.3300816885844
2020-08-10: 2032.6146925705355
2020-08-11: 1832.3426845110394
2020-08-12: 1675.7347393824039
2020-08-13: 1819.9887728638864
2020-08-14: 2015.6223036147233
2020-08-15: 2167.1334030573944
2020-08-16: 2178.4903124340954
2020-08-17: 2089.49862513255
2020-08-18: 2057.421464481878
2020-08-19: 2154.139918975095
2020-08-20: 2291.9501130945546
2020-08-21: 2392.3114031897776
2020-08-22: 2412.2502725470167
2020-08-23: 2391.773320719899
2020-08-24: 2409.8556654520276
2020-08-25: 2491.2928760145205
2020-08-26: 2594.492112488584
2020-08

2020-08-22: 1686.5330575525354
2020-08-23: 1643.7970166243022
2020-08-24: 1654.4120886101487
2020-08-25: 1737.0952403958695
2020-08-26: 1838.357037240618
2020-08-27: 1892.1821635630454
2020-08-28: 1900.6236736033097
2020-08-29: 1900.165907342332
2020-08-30: 1932.6966209690463
2020-08-31: 2002.3207192774285

Predicting for United States__South Carolina
2020-08-01: 1459.9802705230268
2020-08-02: 1392.906723774127
2020-08-03: 1539.0339671791787
2020-08-04: 1647.6002951939117
2020-08-05: 1450.0543835252429
2020-08-06: 1086.1159562923626
2020-08-07: 1472.2163619498024
2020-08-08: 1595.6240627883092
2020-08-09: 1716.4857736360755
2020-08-10: 1734.7473403793997
2020-08-11: 1606.1060878466146
2020-08-12: 1546.0073502549967
2020-08-13: 1697.4052358787299
2020-08-14: 1817.4023096771243
2020-08-15: 1902.759994405609
2020-08-16: 1908.4960865398555
2020-08-17: 1864.1171727615979
2020-08-18: 1876.7494398816461
2020-08-19: 1968.9459718781773
2020-08-20: 2063.8120754592746
2020-08-21: 2128.50501689341

2020-08-19: 1369.5588769337264
2020-08-20: 1443.2046329229465
2020-08-21: 1484.9544154179857
2020-08-22: 1493.3611127070712
2020-08-23: 1497.8504554052683
2020-08-24: 1532.3502240551973
2020-08-25: 1593.3337898024286
2020-08-26: 1652.398345753921
2020-08-27: 1686.490599057042
2020-08-28: 1708.4439375268812
2020-08-29: 1732.3693163971225
2020-08-30: 1773.4514413060028
2020-08-31: 1827.4036465587005

Predicting for United States__Wyoming
2020-08-01: 430.0496880531549
2020-08-02: 628.7041959595417
2020-08-03: 732.9696326703051
2020-08-04: 678.868241695197
2020-08-05: 582.8892686236511
2020-08-06: 430.5291699842522
2020-08-07: 586.7435599839994
2020-08-08: 722.6250221987486
2020-08-09: 788.7308602425148
2020-08-10: 766.4957275270585
2020-08-11: 700.3024664656762
2020-08-12: 673.4728505684093
2020-08-13: 756.0812937321243
2020-08-14: 844.5112471459834
2020-08-15: 890.4925080164971
2020-08-16: 884.4806957311737
2020-08-17: 860.6985046255659
2020-08-18: 870.7302922547094
2020-08-19: 927.85016

CPU times: user 15.9 s, sys: 4.33 s, total: 20.2 s
Wall time: 18.8 s


In [22]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,67.888193
214,Aruba,NaN,2020-08-02,73.163723
215,Aruba,NaN,2020-08-03,84.950810
216,Aruba,NaN,2020-08-04,96.297606
217,Aruba,NaN,2020-08-05,97.853871


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [23]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [24]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,67.88819334179738
Aruba,,2020-08-02,73.1637233995244
Aruba,,2020-08-03,84.95080954976001
Aruba,,2020-08-04,96.29760644457569
Afghanistan,,2020-08-01,221.55244153541952
Afghanistan,,2020-08-02,283.13901472200837
Afghanistan,,2020-08-03,336.41679497414344
Afghanistan,,2020-08-04,399.5451746296149
Angola,,2020-08-01,102.16466310293828


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [25]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [26]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="./predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to ./predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [27]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
CPU times: user 41 ms, sys: 17.1 ms, total: 58.2 ms
Wall time: 1.97 s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [28]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-10
End date: 2021-06-08


In [29]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [30]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-10 to 2021-06-08...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
CPU times: user 3.55 s, sys: 820 ms, total: 4.37 s
Wall time: 3min 7s
